In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import requests
from tqdm import tqdm
import pickle

paePathways = requests.get("http://rest.kegg.jp/link/pathway/pae").text
pathDict = {}
for line in paePathways.strip().split("\n"):
    try:
        loctag, path = line.strip().split("\t")  # Fixed: correct order
        loctag = loctag.replace("pae:", "")
        path = path.replace("path:", "")
        if path not in pathDict.keys():
            pathDict[path] = []
        pathDict[path].append(loctag)
    except ValueError:
        continue

with open("pae_kegg_pathways.gmt", "w") as gmt:
    for pw in pathDict:
        description = f"http://www.kegg.jp/dbget-bin/www_bget?{pw}"
        genes = pathDict[pw]
        gmt.write(f"{pw}\t{description}\t" + "\t".join(genes) + "\n")

# Check the GMT file
with open("pae_kegg_pathways.gmt", "r") as gmt:
    for _ in range(5):  # Print first 5 lines
        print(gmt.readline().strip())

pae00010	http://www.kegg.jp/dbget-bin/www_bget?pae00010	PA0552	PA0555	PA0887	PA1017	PA1498	PA1587	PA1770	PA1982	PA1984	PA2250	PA2266	PA2275	PA2323	PA2555	PA3001	PA3193	PA3195	PA3629	PA3635	PA4022	PA4152	PA4329	PA4732	PA4733	PA4748	PA4829	PA4899	PA4978	PA5015	PA5016	PA5110	PA5131	PA5192	PA5322	PA5422	PA5427
pae00020	http://www.kegg.jp/dbget-bin/www_bget?pae00020	PA0795	PA0854	PA1562	PA1580	PA1581	PA1582	PA1583	PA1584	PA1585	PA1586	PA1587	PA1588	PA1589	PA1787	PA2250	PA2623	PA2624	PA3452	PA4152	PA4333	PA4470	PA4640	PA4829	PA5015	PA5016	PA5192	PA5435	PA5436	PA5445
pae00030	http://www.kegg.jp/dbget-bin/www_bget?pae00030	PA0330	PA0548	PA0555	PA0607	PA0849	PA1499	PA1950	PA2261	PA2263	PA2264	PA2265	PA2290	PA2321	PA2323	PA2796	PA3131	PA3181	PA3182	PA3183	PA3194	PA3373	PA3896	PA4204	PA4670	PA4732	PA5110	PA5322	PA5439
pae00040	http://www.kegg.jp/dbget-bin/www_bget?pae00040	PA0224	PA0607	PA1253	PA2022	PA2023	PA2217	PA2343	PA3559
pae00051	http://www.kegg.jp/dbget-bin/www_bget?pae00051	PA0224	PA0555

In [2]:
# Get set annotation

pathDef = {}

for i in tqdm(pathDict.keys()):
    url = f"http://rest.kegg.jp/get/{i}"
    res = requests.get(url)
    for line in res.text.splitlines():
        if line.startswith("NAME"):
            pathDef[i] = line.replace("NAME", "").strip().replace(" - Pseudomonas aeruginosa PAO1", "")
            break

with open('pae_descriptions.pkl', 'wb') as f:
    pickle.dump(pathDef, f)

100%|██████████| 136/136 [03:34<00:00,  1.58s/it]
